In [550]:
import pandas as pd
import glob
from  geopy.geocoders import Nominatim
import requests
import json
import qwikidata
import qwikidata.sparql


from requests import get
from bs4 import BeautifulSoup
import re
from IPython.display import display_html


In [450]:
#os.chdir('/Users/karlijnrozestraten/Documents/Development.nosync')

#all_files = os.listdir('Woonplaatsen_NL')

#li = []

#for filename in all_files:
  #  df = pd.read_csv('Woonplaatsen_NL/' + filename,sep=';', header=1)
   # df.drop(df.tail(1).index,inplace=True)
    #li.append(df)

#city_names = pd.concat(li, axis=0, ignore_index=True)



#city_names.to_csv('city_names.csv')


In [1000]:
# from: https://opendata.cbs.nl/#/CBS/nl/dataset/84734NED/table
# select alle places
cities = pd.read_csv('Woonplaatsen_NL_complete.csv', sep=';')
cities = cities.set_axis(['Woonplaatsen','Gemeente', 'Provincie','Landsdeel'], axis=1, inplace=False)
cities['Provincie'] = cities['Provincie'].str.replace(' ','')


0          Overijssel
1             Zeeland
2             Drenthe
3       Noord-Holland
4       Noord-Holland
            ...      
2495        Friesland
2496          Drenthe
2497     Zuid-Holland
2498          Drenthe
2499       Overijssel
Name: Provincie, Length: 2500, dtype: object

In [618]:
cities.columns =['Woonplaatsen','Gemeente','Provincie','Landsdeel']
cities.head()

AttributeError: 'list' object has no attribute 'columns'

In [589]:
cities['Woonplaatsen'][:10]

0             Aadorp
1         Aagtekerke
2             Aalden
3           Aalsmeer
4    Aalsmeerderbrug
5              Aalst
6             Aalsum
7             Aalten
8            Ter Aar
9           Ter Aard
Name: Woonplaatsen, dtype: object

In [225]:
# got this function from stack, allows us to get wikidata

def get_city_wikidata(city, country):
    query = """
    SELECT ?city ?cityLabel ?country ?countryLabel ?population ?coordinates
    WHERE
    {
      ?city rdfs:label '%s'@en.
      ?city wdt:P1082 ?population.
      ?city wdt:P17 ?country.
      ?city rdfs:label ?cityLabel.
      ?country rdfs:label ?countryLabel.
      ?city wdt:P625  ?coordinates.
      FILTER(LANG(?cityLabel) = "en").
      FILTER(LANG(?countryLabel) = "en").
      FILTER(CONTAINS(?countryLabel, "%s")).
    }
    """ % (city, country)

    res = qwikidata.sparql.return_sparql_query_results(query)
    out = res['results']['bindings'][0]
    return out



In [324]:
# input for funcution is all the places in the column 'Woonplaats' of the cities df

def get_city_info(city_df):
    
    #cities = pd.read_csv("WoonplaatsenNL.csv", sep=";")
    
    #lijst = list(cities['Woonplaats'])
    
    pop = []
    long = []
    lat = []
    for place in city_df['Woonplaatsen']:
        # the place has a whitespace after the name, remove that first
        place = place.rstrip()
        
        try:
            info = get_city_wikidata(place, 'Netherlands')
            popu_info = info['population']
            popu = int(popu_info.get('value'))
            pop.append(popu)
            longlat_info = info.get('coordinates')
            longlat = longlat_info.get('value')
            longlat = longlat.replace('Point(', '')
            longlat = longlat.replace(')','')
            cor = longlat.split()
            long.append(float(cor[0]))
            lat.append(float(cor[1]))

        except:
            pop.append('No info')
            
            try:
                info = get_city_wikidata(place, 'Netherlands')
                longlat_info = info.get('coordinates')
                longlat = longlat_info.get('value')
                longlat = longlat.replace('Point(', '')
                longlat = longlat.replace(')','')
                cor = longlat.split()
                long.append(float(cor[0]))
                lat.append(float(cor[1]))
            except:
                long.append('No info')
                lat.append('No info')
    
    #print(pop)
    #print(lat)
    #print(long)
    # how the f do I add the listst to the columns of the cities df that's outside this function?
    # or do I need to create the df in this function?
    
    cities["Latitude"] = lat
    cities["Longtitude"] = long
    cities["Population"] = pop
    
    cities.to_csv('City_info.csv')
    
    return cities

        
    

        
    

In [346]:
# input for funcution is all the places in the column 'Woonplaats' of the cities df

def get_city_info(city_df):
    
    #cities = pd.read_csv("WoonplaatsenNL.csv", sep=";")
    
    #lijst = list(cities['Woonplaats'])
    
    pop = []
    long = []
    lat = []
    for place in city_df['Woonplaatsen']:
        print(place)
        try:
            info = get_city_wikidata(place, 'Netherlands')
            
            try:
                popu_info = info['population']
                popu = int(popu_info.get('value'))
                pop.append(popu)
                
                try:
                    longlat_info = info.get('coordinates')
                    longlat = longlat_info.get('value')
                    longlat = longlat.replace('Point(', '')
                    longlat = longlat.replace(')','')
                    cor = longlat.split()
                    long.append(float(cor[0]))
                    lat.append(float(cor[1]))
                
                except:
                    print('joejoejoe')
                    
            except: 
                print('joejoe')
            
            
        
        except IndexError: 
            print('joe')

        
    #print(pop)
    #print(lat)
    #print(long)
    # how the f do I add the listst to the columns of the cities df that's outside this function?
    # or do I need to create the df in this function?
    
    cities["Latitude"] = lat
    cities["Longtitude"] = long
    cities["Population"] = pop
    
    cities.to_csv('City_info.csv')
    
    return cities

        
    

        
    

In [347]:
get_city_info(cities)


Berkhout
Berlicum
Berltsum
joe
Bern
joe
Best
Beugen
joe
Beuningen
Beuningen Gld
joe
Beusichem
joe
Beutenaken
joe
Beverwijk


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [863]:
get_city_wikidata('Amsterdam', 'Netherlands')

{'city': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q727'},
 'population': {'datatype': 'http://www.w3.org/2001/XMLSchema#decimal',
  'type': 'literal',
  'value': '860124'},
 'coordinates': {'datatype': 'http://www.opengis.net/ont/geosparql#wktLiteral',
  'type': 'literal',
  'value': 'Point(4.9 52.383333333)'},
 'country': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q55'},
 'cityLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Amsterdam'},
 'countryLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Netherlands'}}

In [335]:
for place in cities['Woonplaatsen'][:2]:
    
    # the place has a whitespace after the name, remove that first
    print(place)
    place = place.rstrip()

    pop = []
    long = []
    lat = []

    try:
        info = get_city_wikidata(place, 'Netherlands')
        print(info)
        popu_info = info['population']
        popu = int(popu_info.get('value'))
        pop.append(popu)
        longlat_info = info.get('coordinates')
        longlat = longlat_info.get('value')
        longlat = longlat.replace('Point(', '')
        longlat = longlat.replace(')','')
        cor = longlat.split()
        long.append(float(cor[0]))
        lat.append(float(cor[1]))

    except:
        pop.append('No info')

        try:
            info = get_city_wikidata(place, 'Netherlands')
            longlat_info = info.get('coordinates')
            longlat = longlat_info.get('value')
            longlat = longlat.replace('Point(', '')
            longlat = longlat.replace(')','')
            cor = longlat.split()
            long.append(float(cor[0]))
            lat.append(float(cor[1]))
        except:
            long.append('No info')
            lat.append('No info')

Berkhout
{'city': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q2272260'}, 'population': {'datatype': 'http://www.w3.org/2001/XMLSchema#decimal', 'type': 'literal', 'value': '1018'}, 'coordinates': {'datatype': 'http://www.opengis.net/ont/geosparql#wktLiteral', 'type': 'literal', 'value': 'Point(4.9969 52.6419)'}, 'country': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q55'}, 'cityLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Berkhout'}, 'countryLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Netherlands'}}
Berlicum
{'city': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q2897233'}, 'population': {'datatype': 'http://www.w3.org/2001/XMLSchema#decimal', 'type': 'literal', 'value': '1778'}, 'coordinates': {'datatype': 'http://www.opengis.net/ont/geosparql#wktLiteral', 'type': 'literal', 'value': 'Point(5.4 51.6772)'}, 'country': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q55'}, 'cityLabel': {'xml:lang': 'en', 'type': 'lite

In [203]:
# using cities['Woonplaats'] doesn't work as input so I created a list. Maybe I'll add this to the function

lijst = list(cities['Woonplaats'])
print(lijst[11].strip())

Alphen aan den Rijn


In [211]:
# add the lists of function to the cities df

cities["Latitude"] = lat
cities["Longtitude"] = long
cities["Population"] = pop
cities.head(10)

ValueError: Length of values does not match length of index

In [ ]:
# Backup for long lat if not all is gathered
# alsof unction from stack btw
geolocator = Nominatim(user_agent="http")
country ="NL"

lat = []
long = []

for place in cities['Woonplaats']:
    city = place
    loc = geolocator.geocode(city+','+ country)
    lat.append(loc.latitude)
    long.append(loc.longitude)
    print(lat,long)
#print(city, "latitude is :-" ,loc.latitude,"\nlongtitude is:-" ,loc.longitude)

#cities.loc[city] = 

cities["Latitude"] = lat
cities["Longtitude"] = long

In [183]:
# create dataframe from the Dutch places file

#cities = pd.read_csv("WoonplaatsenNL.csv", sep=";")
#cities.head()


,ID,Woonplaatsen,Woonplaatscode_1,Woonplaats,Code_3,Provincie,Code_5,Deel,Code_7
0,0,WP1925,WP1925,Almelo,GM0141,Overijssel,PV23,Oost-Nederland,LD02
1,1,WP2774,WP2774,Veere,GM0717,Zeeland,PV29,West-Nederland,LD03
2,2,WP2145,WP2145,Coevorden,GM0109,Drenthe,PV22,Noord-Nederland,LD01
3,3,WP2571,WP2571,Aalsmeer,GM0358,Noord-Holland,PV27,West-Nederland,LD03
4,4,WP1600,WP1600,Haarlemmermeer,GM0394,Noord-Holland,PV27,West-Nederland,LD03


In [348]:
def get_city_opendata(city, country):
    tmp = 'https://public.opendatasoft.com/api/records/1.0/search/?dataset=worldcitiespop&q=%s&sort=population&facet=country&refine.country=%s'
    cmd = tmp % (city, country)
    res = requests.get(cmd)
    dct = json.loads(res.content)
    out = dct['records'][0]['fields']
    return out

In [349]:
for city in cities[:10]:
    print(get_city_opendata(city, 'nl'))

KeyError: 'records'

{'city': 'zwanenburg',
 'country': 'nl',
 'region': '07',
 'geopoint': [52.374819, 4.731293],
 'longitude': 4.731293,
 'latitude': 52.374819,
 'accentcity': 'Zwanenburg'}

In [1037]:
response = get('https://nl.wikipedia.org/wiki/Categorie:Lijsten_van_Nederlandse_plaatsen_naar_provincie')
html_soup = BeautifulSoup(response.text, 'html.parser')
data = html_soup.find('div',{'class':'mw-category'})

links = data.findAll('a')
provinces_links = []
for link in links:
    if link['href'][6:22] == 'Lijst_van_steden':
        plink = 'https://nl.wikipedia.org'+link['href']
        provinces_links.append(plink)

provinces = cities['Provincie'].unique()

for province in provinces:
    pl = re.compile('.*\D({})$'.format(province))
    for find_link in provinces_links:
        m = pl.match(find_link)
        if m:
            print(province)
            print(find_link)

#places_in_province = cities.loc[cities['Provincie'] == province, 'Woonplaatsen']



Overijssel
https://nl.wikipedia.org/wiki/Lijst_van_steden_en_dorpen_in_Overijssel
Zeeland
https://nl.wikipedia.org/wiki/Lijst_van_steden_en_dorpen_in_Zeeland
Drenthe
https://nl.wikipedia.org/wiki/Lijst_van_steden_en_dorpen_in_Drenthe
Noord-Holland
https://nl.wikipedia.org/wiki/Lijst_van_steden_en_dorpen_in_Noord-Holland
Gelderland
https://nl.wikipedia.org/wiki/Lijst_van_steden_en_dorpen_in_Gelderland
Friesland
https://nl.wikipedia.org/wiki/Lijst_van_steden_en_dorpen_in_Friesland
Zuid-Holland
https://nl.wikipedia.org/wiki/Lijst_van_steden_en_dorpen_in_Zuid-Holland
Noord-Brabant
https://nl.wikipedia.org/wiki/Lijst_van_steden_en_dorpen_in_Noord-Brabant
Groningen
https://nl.wikipedia.org/wiki/Lijst_van_steden_en_dorpen_in_Groningen
Flevoland
https://nl.wikipedia.org/wiki/Lijst_van_steden_en_dorpen_in_Flevoland


In [1065]:
#province = 'Drenthe'
#place = 'Aalden'
#province = 'Noord-Holland'
#place = 'Aagtdorp'
#link_list = []

no_places = ['Basse', 'Bruinehaar', 'Collendoorn', 'Delden','Driewegen', 'Hem', 'Holten', 'Ressen', 'Morra','Riel','Houwerzijl','Onstwedde']               
# create dataframe for output
results = pd.DataFrame(columns = ['Place','Link','Province','Coordinates', 'Population', 'Lattitude', 'Longitude'])


# retrieve information from wikipedia page about provinces
response = get('https://nl.wikipedia.org/wiki/Categorie:Lijsten_van_Nederlandse_plaatsen_naar_provincie')
html_soup = BeautifulSoup(response.text, 'html.parser')
data = html_soup.find('div',{'class':'mw-category'})

# retrieve the correct links for the pages with all places per province
links = data.findAll('a')
provinces_links = []
for link in links:
    if link['href'][6:22] == 'Lijst_van_steden':
        plink = 'https://nl.wikipedia.org'+link['href']
        provinces_links.append(plink)
# create a list of all the unique provinces to make iteration possible
provinces = cities['Provincie'].unique()

# start the iteration for each provinces
for province in provinces:
    item_list = []
   
    # retrieve the overview page of the province
    pl = re.compile('.*\D({})$'.format(province))
    for find_link in provinces_links:
        m = pl.match(find_link)

        if m:
            province_page = get(find_link)
            html_soup = BeautifulSoup(province_page.text, 'html.parser')        
            # expend the iteration by looping for each place in a province. Create a list of the places in a province
            places_in_province = cities.loc[cities['Provincie'] == province, 'Woonplaatsen']
            for place in places_in_province:
                if place not in no_places:
                    #print(place)
                    print(province, place)
                    # The format of the province Drenthe is slightly different then the other provinces. The approach differs slightly
                    if province == 'Drenthe':
                        # Only retrieve the elements where the information we need is in
                        page_data = html_soup.findAll('table',{'class':''})
                        tr_list = [tr.findAll('tr') for tr in page_data]
                        place_link_list = [li.find('a') for item in tr_list for li in item if li.find('a')!= None]

                        # Get the link for the place page in order to retrieve information about the place
                        for place_link in place_link_list:
                            if place_link != None:
                                if p['href'][6:(len(place)+6)] == place:
                                    
                                    placelink = 'https://nl.wikipedia.org'+place_link['href']
                                    print(place_link)
                                    # Get access to the infobox on the pace
                                    place_page = get(placelink)
                                    html_soup = BeautifulSoup(place_page.text, 'html.parser')
                                    place_data = html_soup.find('table',{'class':'infobox'})
                                    # Retrieve the coordinate and population information
                                    table_df = pd.read_html(str(place_data))[0]
                                    co = table_df.loc[table_df[0] == 'Coördinaten',[1]].values[0]
                                    pop = table_df[table_df[0].str.match(r'Inwoners.*')==True][1].values[0]
                                   
                                    # Transform the coordinates to decimals
                                    split = str(co).rsplit(',')
                                    lat = split[0]
                                    long = split[1]
                                    lat_ = re.findall('\d+', lat)
                                    long_ = re.findall('\d+',long)
                                    lat_ = [int(c) for c in lat_]
                                    long_ = [int(c) for c in long_]

                                    if len(lat_) != 3:
                                        lat_.append(0)
                                    latdd = lat_[0] + (lat_[1]/60) + (lat_[2]/3600)

                                    if len(long_) !=3:
                                        long_.append(0)
                                    longdd = long_[0] + (long_[1]/60) + (long_[2]/3600)
                                    # Add the information to the dataframe
                                    results.loc[len(results)] = [place, p, province, co, pop, latdd, longdd]
                                    #print(results)       

                    else:
                        # Retrieve information form the province overview page
                        page_data = html_soup.find('div',{'class':'mw-parser-output'})
                        tr_list = page_data.findAll('tr')
                        # Create a list with the places in the province and the link to it
                        for item in tr_list:
                            item_list.append(item.findAll('li'))   
                        places_list = [li.find('a') for li_list in item_list for li in li_list if li != None]
                        # Get the link for the place page in order to retrieve information about the place
                        for place_link in places_list:
                            if place_link != None and place_link['href'][6:] != 'Baarlo_(Zwartewaterland)':
                                #print(place_link['href'][6:])
                                if place_link['href'][6:(len(place)+6)] == place:
                                    placelink = 'https://nl.wikipedia.org'+place_link['href']
                                    print(place_link)
                                    # Get access to the infobox on the pace
                                    place_page = get(placelink)
                                    html_soup = BeautifulSoup(place_page.text, 'html.parser')
                                    place_data = html_soup.find('table',{'class':'infobox'})
                                    # Retrieve the coordinate and population information
                                    table_df = pd.read_html(str(place_data))[0]
                                    co = table_df.loc[table_df[0] == 'Coördinaten',[1]].values[0]
                                    pop = table_df[table_df[0].str.match(r'Inwoners.*')==True][1].values[0]
                                
                                    # Transform the coordinates to decimals
                                    split = str(co).rsplit(',')
                                    lat = split[0]
                                    long = split[1]
                                    lat_ = re.findall('\d+', lat)
                                    long_ = re.findall('\d+',long)
                                    lat_ = [int(c) for c in lat_]
                                    long_ = [int(c) for c in long_]

                                    if len(lat_) != 3:
                                        lat_.append(0)
                                    latdd = lat_[0] + (lat_[1]/60) + (lat_[2]/3600)

                                    if len(long_) !=3:
                                        long_.append(0)
                                    longdd = long_[0] + (long_[1]/60) + (long_[2]/3600)
                                    # Add the information to the dataframe
                                    results.loc[len(results)] = [place, place_link, province, co, pop, latdd, longdd]
                                    #print(results)

results.to_csv('results.csv')
print(results)




Overijssel Aadorp
<a href="/wiki/Aadorp" title="Aadorp">Aadorp</a>
Overijssel Agelo
<a href="/wiki/Agelo" title="Agelo">Agelo</a>
Overijssel Albergen
<a href="/wiki/Albergen" title="Albergen">Albergen</a>
Overijssel Almelo
<a href="/wiki/Almelo" title="Almelo">Almelo</a>
Overijssel Ambt Delden
Overijssel Ane
<a href="/wiki/Ane" title="Ane">Ane</a>
<a href="/wiki/Anerveen" title="Anerveen">Anerveen</a>
<a href="/wiki/Anevelde" title="Anevelde">Anevelde</a>
Overijssel Anerveen
<a href="/wiki/Anerveen" title="Anerveen">Anerveen</a>
Overijssel Anevelde
<a href="/wiki/Anevelde" title="Anevelde">Anevelde</a>
Overijssel Arriën
Overijssel Baarlo
<a href="/wiki/Baarlo_(Steenwijkerland)" title="Baarlo (Steenwijkerland)">Baarlo</a>
Overijssel Baars
<a href="/wiki/Baars_(gehucht)" title="Baars (gehucht)">Baars</a>
Overijssel Balkbrug
<a href="/wiki/Balkbrug_(Hardenberg)" title="Balkbrug (Hardenberg)">Balkbrug</a>
Overijssel Bathmen
<a href="/wiki/Bathmen" title="Bathmen">Bathmen</a>
Overijssel Bee

<a href="/wiki/Lutten-Oever" title="Lutten-Oever">Lutten-Oever</a>


IndexError: index 0 is out of bounds for axis 0 with size 0

In [1051]:
#results.tail()

results.loc[results['Province']=='Drenthe']

,Place,Link,Province,Coordinates,Population,Lattitude,Longitude


In [911]:
province = 'Noord-Holland'
place = 'Bergen'
links = []

places_in_province = cities.loc[cities['Provincie'] == province, 'Woonplaatsen']
pl = re.compile('.*\D({})$'.format(province))
for find_link in provinces_links:
    m = pl.match(find_link)
    if m:
        province_page = get(find_link)
        html_soup = BeautifulSoup(province_page.text, 'html.parser')
        page_data = html_soup.find('div',{'class':'mw-parser-output'})
        places_links = page_data.findAll('tr')
        for l in places_links:
            if l.find('a') != None:
                links.append(l)
        for i in links:
            linkie = i.findAll('a')
            for k in linkie:
                #print(k)
                if k['href'][6:(len(place)+6)] == place:
                    print(k['href'])
                    placelink = 'https://nl.wikipedia.org'+k['href']
                    place_page = get(placelink)
                    html_soup = BeautifulSoup(place_page.text, 'html.parser')
                    place_data = html_soup.find('table',{'class':'infobox'})

                    table = pd.read_html(str(place_data))
                    table_df = table[0]

                    co = table_df[1][5]
                    pop = table_df[1][7]
                    print(co,pop)
                else:
                    pass
               


                    
                    


<a href="/wiki/Aagtdorp" title="Aagtdorp">Aagtdorp</a>
<a href="/wiki/Bergen_(Noord-Holland)" title="Bergen (Noord-Holland)">Bergen</a>
/wiki/Bergen_(Noord-Holland)
Situering 52° 40′ NB, 4° 42′ OL
<a href="/wiki/Aalsmeer" title="Aalsmeer">Aalsmeer</a>
<a href="/wiki/Aalsmeer" title="Aalsmeer">Aalsmeer</a>
<a class="mw-redirect" href="/wiki/Aalsmeerderbrug_(buurtschap)" title="Aalsmeerderbrug (buurtschap)">Aalsmeerderbrug</a>
<a href="/wiki/Haarlemmermeer" title="Haarlemmermeer">Haarlemmermeer</a>
<a href="/wiki/Aartswoud" title="Aartswoud">Aartswoud</a>
<a href="/wiki/Opmeer" title="Opmeer">Opmeer</a>
<a href="/wiki/Abbekerk" title="Abbekerk">Abbekerk</a>
<a href="/wiki/Medemblik_(gemeente)" title="Medemblik (gemeente)">Medemblik</a>
<a href="/wiki/Abbekerkeweere" title="Abbekerkeweere">Abbekerkeweere</a>
<a href="/wiki/Opmeer" title="Opmeer">Opmeer</a>
<a href="/wiki/Abbenes" title="Abbenes">Abbenes</a>
<a href="/wiki/Haarlemmermeer" title="Haarlemmermeer">Haarlemmermeer</a>
<a href="

Situering 52° 40′ NB, 4° 42′ OL
<a href="/wiki/Breukeleveen" title="Breukeleveen">Breukeleveen</a>
<a href="/wiki/Wijdemeren" title="Wijdemeren">Wijdemeren</a>
<a href="/wiki/Broek_in_Waterland" title="Broek in Waterland">Broek in Waterland</a>
<a href="/wiki/Waterland_(gemeente)" title="Waterland (gemeente)">Waterland</a>
<a href="/wiki/Broek_op_Langedijk" title="Broek op Langedijk">Broek op Langedijk</a>
<a href="/wiki/Langedijk" title="Langedijk">Langedijk</a>
<a href="/wiki/Broekhorn" title="Broekhorn">Broekhorn</a>
<a href="/wiki/Heerhugowaard" title="Heerhugowaard">Heerhugowaard</a>
<a href="/wiki/Broerdijk" title="Broerdijk">Broerdijk</a>
<a href="/wiki/Medemblik_(gemeente)" title="Medemblik (gemeente)">Medemblik</a>
<a href="/wiki/Buiksloot" title="Buiksloot">Buiksloot</a>
<a href="/wiki/Amsterdam_(gemeente)" title="Amsterdam (gemeente)">Amsterdam</a>
<a href="/wiki/Buitenkaag" title="Buitenkaag">Buitenkaag</a>
<a href="/wiki/Haarlemmermeer" title="Haarlemmermeer">Haarlemmermee

Situering 52° 40′ NB, 4° 42′ OL
<a href="/wiki/Edam" title="Edam">Edam</a>
<a href="/wiki/Edam-Volendam" title="Edam-Volendam">Edam-Volendam</a>
<a href="/wiki/Eenigenburg" title="Eenigenburg">Eenigenburg</a>
<a href="/wiki/Schagen_(gemeente)" title="Schagen (gemeente)">Schagen</a>
<a href="/wiki/Egelshoek" title="Egelshoek">Egelshoek</a>
<a href="/wiki/Hilversum" title="Hilversum">Hilversum</a>
<a href="/wiki/Egmond_aan_den_Hoef" title="Egmond aan den Hoef">Egmond aan den Hoef</a>
<a href="/wiki/Bergen_(Noord-Holland)" title="Bergen (Noord-Holland)">Bergen</a>
/wiki/Bergen_(Noord-Holland)
Situering 52° 40′ NB, 4° 42′ OL
<a href="/wiki/Egmond_aan_Zee" title="Egmond aan Zee">Egmond aan Zee</a>
<a href="/wiki/Bergen_(Noord-Holland)" title="Bergen (Noord-Holland)">Bergen</a>
/wiki/Bergen_(Noord-Holland)
Situering 52° 40′ NB, 4° 42′ OL
<a href="/wiki/Egmond-Binnen" title="Egmond-Binnen">Egmond-Binnen</a>
<a href="/wiki/Bergen_(Noord-Holland)" title="Bergen (Noord-Holland)">Bergen</a>
/wiki

<a href="/wiki/Hollands_Kroon_(gemeente)" title="Hollands Kroon (gemeente)">Hollands Kroon</a>
<a href="/wiki/De_Kolk_(Opmeer)" title="De Kolk (Opmeer)">De Kolk</a>
<a href="/wiki/Opmeer" title="Opmeer">Opmeer</a>
<a href="/wiki/De_Koog" title="De Koog">De Koog</a>
<a href="/wiki/Texel" title="Texel">Texel</a>
<a href="/wiki/Koog_aan_de_Zaan" title="Koog aan de Zaan">Koog aan de Zaan</a>
<a href="/wiki/Zaanstad" title="Zaanstad">Zaanstad</a>
<a href="/wiki/De_Kooy" title="De Kooy">De Kooy</a>
<a href="/wiki/Den_Helder" title="Den Helder">Den Helder</a>
<a href="/wiki/Het_Korfwater" title="Het Korfwater">Het Korfwater</a>
<a href="/wiki/Schagen_(gemeente)" title="Schagen (gemeente)">Schagen</a>
<a href="/wiki/Kortenhoef" title="Kortenhoef">Kortenhoef</a>
<a href="/wiki/Wijdemeren" title="Wijdemeren">Wijdemeren</a>
<a href="/wiki/Kraaienburg" title="Kraaienburg">Kraaienburg</a>
<a href="/wiki/Drechterland_(gemeente)" title="Drechterland (gemeente)">Drechterland</a>
<a href="/wiki/Krabben

Situering 52° 40′ NB, 4° 42′ OL
<a href="/wiki/Schoorldam" title="Schoorldam">Schoorldam</a>
<a href="/wiki/Bergen_(Noord-Holland)" title="Bergen (Noord-Holland)">Bergen</a>
/wiki/Bergen_(Noord-Holland)
Situering 52° 40′ NB, 4° 42′ OL
<a href="/wiki/Schoorldam" title="Schoorldam">Schoorldam</a>
<a href="/wiki/Schagen_(gemeente)" title="Schagen (gemeente)">Schagen</a>
<a href="/wiki/Het_Schouw" title="Het Schouw">Het Schouw</a>
<a href="/wiki/Waterland_(gemeente)" title="Waterland (gemeente)">Waterland</a>
<a href="/wiki/Amsterdam_(gemeente)" title="Amsterdam (gemeente)">Amsterdam</a>
<a href="/wiki/Schulpstet" title="Schulpstet">Schulpstet</a>
<a href="/wiki/Castricum" title="Castricum">Castricum</a>
<a href="/wiki/%27s-Graveland_(Noord-Holland)" title="'s-Graveland (Noord-Holland)">'s-Graveland</a>
<a href="/wiki/Wijdemeren" title="Wijdemeren">Wijdemeren</a>
<a href="/wiki/Sijbekarspel" title="Sijbekarspel">Sijbekarspel</a>
<a href="/wiki/Medemblik_(gemeente)" title="Medemblik (gemeen

Situering 52° 40′ NB, 4° 42′ OL
<a href="/wiki/De_Woude" title="De Woude">De Woude</a>
<a href="/wiki/Castricum" title="Castricum">Castricum</a>
<a href="/wiki/Woudmeer" title="Woudmeer">Woudmeer</a>
<a href="/wiki/Schagen_(gemeente)" title="Schagen (gemeente)">Schagen</a>
<a class="mw-redirect" href="/wiki/Zaandam_(hoofdbetekenis)" title="Zaandam (hoofdbetekenis)">Zaandam</a>
<a href="/wiki/Zaanstad" title="Zaanstad">Zaanstad</a>
<a href="/wiki/Zaandijk" title="Zaandijk">Zaandijk</a>
<a href="/wiki/Zaanstad" title="Zaanstad">Zaanstad</a>
<a href="/wiki/Zaanse_Schans" title="Zaanse Schans">Zaanse Schans</a>
<a href="/wiki/Zaanstad" title="Zaanstad">Zaanstad</a>
<a href="/wiki/Zanegeest" title="Zanegeest">Zanegeest</a>
<a href="/wiki/Bergen_(Noord-Holland)" title="Bergen (Noord-Holland)">Bergen</a>
/wiki/Bergen_(Noord-Holland)
Situering 52° 40′ NB, 4° 42′ OL
<a href="/wiki/%27t_Zand_(Schagen)" title="'t Zand (Schagen)">'t Zand</a>
<a href="/wiki/Schagen_(gemeente)" title="Schagen (gemee

KeyError: 'href'

In [642]:
print(cities.loc[cities['Provincie'] == 'Noord-Holland', 'Woonplaatsen'])

Series([], Name: Woonplaatsen, dtype: object)


In [459]:
provinces = cities['Provincie'].unique()
for province in provinces[:2]:
    places_in_province = cities.loc[cities['Provincie'] == province, 'Woonplaatsen']
    for place in places_in_province:
        

0           Berkhout
10         Beverwijk
25          Blaricum
36       Bloemendaal
37           Blokker
            ...     
2441     Callantsoog
2446       Castricum
2449    De Cocksdorp
2458        Cruquius
2494          Diemen
Name: Woonplaatsen, Length: 238, dtype: object
1                 Berlicum
4                     Best
5                   Beugen
15         Biest-Houtakker
16            Biezenmortel
               ...        
2484                 Demen
2486                Deurne
2489    Deursen-Dennenburg
2493                Dieden
2498               Diessen
Name: Woonplaatsen, Length: 279, dtype: object


In [592]:
#cities.isnull().sum().sum()

is_NaN = cities.isnull()
row_has_NaN = is_NaN.any(axis=1)
rows_with_NaN = cities[row_has_NaN]

rows_with_NaN

,Woonplaatsen,Gemeente/Naam (Naam),Provincie/Naam (Naam),Landsdeel/Naam (Naam)


In [ ]:
for provincie in lijst provincies
    maak lijst van plaatsen per provincie
    ga naar de pagina van de provincie
    ga naar pagina van de plaats
    lees de infobox uit
    voeg dit toe aan de dataframe